In [2]:
# !wget https://raw.githubusercontent.com/sugarforever/Finetune-GPT-Model/main/dataset.jsonl
# !wget crab233.cloudns.biz/proxy/raw.githubusercontent.com/sugarforever/Finetune-GPT-Model/main/dataset.jsonl

--2024-06-28 13:54:16--  http://crab233.cloudns.biz/proxy/raw.githubusercontent.com/sugarforever/Finetune-GPT-Model/main/dataset.jsonl
Resolving crab233.cloudns.biz (crab233.cloudns.biz)... 76.76.21.21
Connecting to crab233.cloudns.biz (crab233.cloudns.biz)|76.76.21.21|:80... connected.
HTTP request sent, awaiting response... 308 Permanent Redirect
Location: https://crab233.cloudns.biz/proxy/raw.githubusercontent.com/sugarforever/Finetune-GPT-Model/main/dataset.jsonl [following]
--2024-06-28 13:54:17--  https://crab233.cloudns.biz/proxy/raw.githubusercontent.com/sugarforever/Finetune-GPT-Model/main/dataset.jsonl
Connecting to crab233.cloudns.biz (crab233.cloudns.biz)|76.76.21.21|:443... connected.
HTTP request sent, awaiting response... 308 Permanent Redirect
Location: https://www.crab233.cloudns.biz/proxy/raw.githubusercontent.com/sugarforever/Finetune-GPT-Model/main/dataset.jsonl [following]
--2024-06-28 13:54:18--  https://www.crab233.cloudns.biz/proxy/raw.githubusercontent.com/suga

In [3]:
# 预处理数据集
!openai tools fine_tunes.prepare_data -f dataset.jsonl -q

Analyzing...

- Your file contains 50 prompt-completion pairs. In general, we recommend having at least a few hundred examples. We've found that performance tends to linearly increase for every doubling of the number of examples
- Based on your data it seems like you're trying to fine-tune a model for classification
- For classification, we recommend you try one of the faster and cheaper models, such as `ada`
- For classification, you can estimate the expected model performance by keeping a held out dataset, which is not used for training
- There are 1 duplicated prompt-completion sets. These are rows: [42]
- All prompts end with suffix ` ->`
- The completion should start with a whitespace character (` `). This tends to produce better results due to the tokenization we use. See https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more details

Based on the analysis we will perform the following actions:
- [Recommended] Remove 1 duplicate rows [Y/n]: Y
- [Recom

In [15]:
# 上传数据集
import os
import openai
from dotenv import load_dotenv

load_dotenv(override=True)
openai.api_key = os.getenv("DASHSCOPE_API_KEY")
openai.base_url = os.getenv("DASHSCOPE_API_BASE")

file = openai.File.create(
    file=open("dataset_prepared_train.jsonl", "rb"),
    purpose='fine-tune'
)

file_id = file.id
print(file_id)

APIRemovedInV1: 

You tried to access openai.File, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [8]:
from pathlib import Path
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv(override=True)

api_key = os.getenv("DASHSCOPE_API_KEY")
base_url = os.getenv("DASHSCOPE_API_BASE")
print(api_key, base_url)

client = OpenAI(
    api_key=api_key,
    base_url=base_url
)

sk-01c5003340c3453b934052d737d45e01 https://dashscope.aliyuncs.com/compatible-mode/v1


In [14]:
# qwen的方式上传数据，上传数据集
file = client.files.create(
    file=open("dataset_prepared_train.jsonl", "rb"),
    purpose='fine-tune'
)

file_id = file.id
print(file_id)

BadRequestError: Error code: 400 - {'error': {'message': "'purpose' must be 'file-extract'", 'type': 'invalid_request_error', 'param': 'purpose', 'code': None}}

In [9]:
# data.pdf 是一个示例文件
file_object = client.files.create(file=Path("dataset_prepared_train.jsonl"), purpose="file-extract")

In [13]:
file_object.id

'file-fe-tvNhgFp95hoyykAWj6tiFfkW'

In [21]:
fine_tune = client.fine_tuning.jobs.create(training_file=file_object.id, model='ada')
fine_tune_id = fine_tune.id

print(fine_tune_id)

NotFoundError: Error code: 404

In [ ]:
# 查看微调任务状态
status = openai.FineTune.retrieve(id=fine_tune_id)
# client.files.retrieve(file_object.id)

print(f"ID: {status.id}")
print(f"Status: {status.status}")
print(f"Model: {status.model}")
print(f"Fine tuned model: {status.fine_tuned_model}")

In [22]:
# 查询（分类提问）
response = openai.Completion.create(
    model="ada:ft-personal-2023-04-24-21-32-47",
    prompt="这么六的吉他弹奏许久不见 ->",
    temperature=0.7,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop=["END"]
)

APIRemovedInV1: 

You tried to access openai.Completion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [ ]:
print(response)
